# VoltAI: Smart Assistant for Your Vehicle

Meet VoltAI, your intelligent assistant designed to keep your vehicle running smoothly and efficiently. Powered by Python, Pandas, and OpenAI's advanced language models, VoltAI brings real-time insights and smart decision-making right to your dashboard.

With specialized agents like:

- 🚗 Vehicle Health Analyst – Monitors engine temperature, load, and overall performance.
- 🚨 Alert Generator – Instantly flags critical issues like low tire pressure or warning lights.
- ⛽ Fuel Efficiency Advisor – Tracks fuel trends to help you save money and go the distance.
- 🔧 Maintenance Predictor – Anticipates upcoming maintenance needs before problems arise.

Just ask:

“Is my engine running hot today?”

“Should I schedule an oil change soon?”

“How’s my tire pressure looking?”

VoltAI turns raw vehicle data into real-time intelligence—so you can drive smarter, safer, and stress-free.

Team 4 Members:
- Abhijith Raj Urs Ravishankar
- Harhshitha Banagalore Ramachandra
- Sanjana Rao

In [37]:
!pip install openai gradio

In [38]:
%%capture
# 🚀 Installing CrewAI, LangChain, OpenAI, and additional tools/libraries quietly...
!pip install -q crewai langchain langchain-openai openai crewai-tools ipython
!pip install 'crewai[tools]'

In [39]:
import os
from google.colab import userdata

# Load OpenAI API key from Colab userdata secrets
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GOOGLE_API_KEY')
# Set default OpenAI model for CrewAI agents
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

# Confirmation message
print("✅ OpenAI key, Gemini Key and model are successfully configured.")

✅ OpenAI key, Gemini Key and model are successfully configured.


In [40]:
!pip install openai gradio pandas

import pandas as pd
import openai
import gradio as gr

In [41]:
# Upload a file manually or handle from other sources in future
uploaded_df = None

def load_data_from_file(file_obj):
    global uploaded_df
    uploaded_df = pd.read_csv(file_obj.name)
    return "✅ File uploaded and data loaded!"
# To load from Google Drive or API
# df = pd.read_csv("https://drive.google.com/uc?id=<file_id>")
# or df = requests.get("your_api_url").json()

In [42]:
def check_engine_temperature(temp):
    if temp > 100:
        return "⚠️ High engine temperature! Immediate attention needed."
    elif temp < 70:
        return "⚠️ Engine temperature lower than usual. Check cooling system."
    return "✅ Engine temperature is normal."

def check_oil_level(oil_level):
    return "⚠️ Oil level is low. Consider an oil change." if oil_level < 50 else "✅ Oil level is good."

def check_fuel_level(fuel_level):
    return "⚠️ Low fuel level. Please refill soon." if fuel_level < 25 else "✅ Fuel level is sufficient."

def check_tire_pressure(row):
    alerts = []
    for tire in ['tire_pressure_FL', 'tire_pressure_FR', 'tire_pressure_RL', 'tire_pressure_RR']:
        if row[tire] < 30:
            alerts.append(f"⚠️ Low pressure in {tire[-2:]} tire.")
    return alerts if alerts else ["✅ All tire pressures are within normal range."]

def check_engine_light(light_on):
    return "⚠️ Check engine light is ON." if light_on else "✅ No engine warning lights."

In [43]:
# Diagnostic summary using last entry
import openai

# New SDK: create OpenAI client
client = openai.OpenAI()

def generate_vehicle_summary(vehicle_id, df):
    try:
        if df is None or df.empty:
            return "❌ Please upload data first."

        if vehicle_id not in df['vehicle_id'].unique():
            return f"❌ Vehicle ID '{vehicle_id}' not found in data."

        latest = df[df['vehicle_id'] == vehicle_id].sort_values("tracking_timestamp").iloc[-1]

        checks = (
            f"🔧 Vehicle Diagnostic Summary\n"
            f"- Engine Temp: {latest['temp_engine_C']} °C\n"
            f"- Fuel Level: {latest['fuel_level_percent']} %\n"
            f"- Tire Pressure FL: {latest['tire_pressure_FL']} psi\n"
            f"- Tire Pressure FR: {latest['tire_pressure_FR']} psi\n"
            f"- Tire Pressure RL: {latest['tire_pressure_RL']} psi\n"
            f"- Tire Pressure RR: {latest['tire_pressure_RR']} psi\n"
            f"- Timestamp: {latest['tracking_timestamp']}"
        )

        prompt = f"""
        You are an expert car advisor. A user has the following vehicle status:

        {checks}

        Please summarize what this means in simple words.
        """

        # ✅ New SDK call
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}]
        )

        ai_summary = response.choices[0].message.content.strip()

        return f"{checks}\n\n🤖 AI Summary:\n{ai_summary}"

    except Exception as e:
        return f"❌ Internal error: {str(e)}"



In [44]:
client = openai.OpenAI()

def query_vehicle_data(user_query, df):
    try:
        if df is None or df.empty:
            return "❌ Please upload the vehicle data first."

        sample_data = df.tail(5).to_csv(index=False)

        prompt = f"""
        You are an expert vehicle data analyst. Here's the recent sensor data from a car:

        {sample_data}

        Now answer the user's question: "{user_query}"
        """

        response = client.chat.completions.create(
            model="gpt-4o",  # or gpt-3.5-turbo
            messages=[
                {"role": "user", "content": prompt}
            ]
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"❌ Internal error: {str(e)}"


In [45]:
import pandas as pd
from crewai import Crew, Agent, Task, Process, LLM
from langchain.llms import OpenAI
from langchain.agents import Tool as LangchainTool # Import Tool from langchain and rename to avoid conflict
import os
import gradio as gr
import openai
import google.generativeai as genai
from dotenv import load_dotenv
from langchain.tools import tool
from IPython.display import display, Markdown
from langchain.llms import OpenAI

llm = LLM(model="gpt-4o")

@tool
def ask_about_specific_vehicle(vehicle_id: str, question: str, df: pd.DataFrame) -> str:
    """Query OpenAI about a specific vehicle ID using its data only."""

    vehicle_row = df[df['vehicle_id'] == vehicle_id]

    if vehicle_row.empty:
        return f"❌ Vehicle ID {vehicle_id} not found in the dataset."

    vehicle_data_str = vehicle_row.to_string(index=False)

    prompt = f"""
You are a smart vehicle diagnostic assistant.

You are analyzing the following data for a single vehicle (Vehicle ID: {vehicle_id}):
{vehicle_data_str}

Now answer this question based ONLY on that vehicle's data:

{question}

Be specific and detailed. Act like a skilled mechanic. For example:
- If the question is about oil level, mention the percentage and give a clear recommendation.
- If it's about tire pressure, mention each tire's pressure and suggest adjustments.
- If it's about engine temperature, mention the exact temperature and suggest precautions.
- If it's about fuel level, mention the percentage and suggest refueling if needed.
- If it's about engine load, mention the percentage and suggest driving style adjustments.
- If it's about air pressure, mention the pressure and suggest checking for leaks.
- If it's about check engine light, say whether it's on and possible reasons.
- Avoid listing all columns or sounding generic. Be a helpful expert mechanic!
"""

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a precise and helpful vehicle assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        return f"OpenAI Error: {str(e)}"

# Define Agents
vehicle_health_analyst = Agent(
    role='Vehicle Health Analyst',
    goal='Analyze vehicle data to determine health status and detect anomalies.',
    backstory='An expert in automotive diagnostics with deep knowledge of onboard telemetry systems.',
    verbose=True,
    llm=llm
)

alert_generator = Agent(
    role='Alert Generator',
    goal='Generate actionable alerts based on anomalies or threshold breaches in vehicle data.',
    backstory='A seasoned system monitor designed to notify users in real-time about potential issues.',
    verbose=True,
    llm=llm
)

fuel_efficiency_advisor = Agent(
    role='Fuel Efficiency Advisor',
    goal='Evaluate driving and engine metrics to provide suggestions for better fuel efficiency.',
    backstory='A fuel economist with insights on optimizing engine performance and driving behavior.',
    verbose=True,
    llm=llm
)

maintenance_predictor = Agent(
    role='Maintenance Predictor',
    goal='Predict upcoming vehicle maintenance needs using past data and operational trends.',
    backstory='A predictive maintenance AI trained on thousands of fleet vehicle histories.',
    verbose=True,
    llm=llm
)

# 1️⃣ Task for the Vehicle Health Analyst agent
health_analysis_task = Task(
    description=(
        "Analyze the vehicle's recent telemetry data including engine temperature, oil level, tire pressure, speed, and engine load. "
        "Detect any abnormal readings or potential early signs of failure."
    ),
    expected_output=(
        "A concise diagnostic summary highlighting vehicle health, including any unusual patterns or values that need attention."
    ),
    agent=vehicle_health_analyst
)

# 2️⃣ Task for the Alert agent
alert_task = Task(
    description=(
        "Review the diagnostic data and identify any urgent issues such as high engine temperature, low tire pressure, "
        "critical oil levels, or check engine light status. Generate warnings or alerts if thresholds are breached."
    ),
    expected_output=(
        "A list of alerts with descriptions and severity levels, highlighting the issues that require immediate driver attention."
    ),
    agent=alert_generator
)

# 3️⃣ Task for the Fuel Efficieny agent
fuel_efficiency_task = Task(
    description=(
        "Assess the vehicle's average speed, fuel level trends, and engine load to estimate fuel efficiency. "
        "Recommend ways the driver can improve mileage and reduce fuel consumption."
    ),
    expected_output=(
        "A breakdown of fuel efficiency metrics and clear tips for improving mileage based on current driving patterns."
    ),
    agent=fuel_efficiency_advisor
)

# 4️⃣  Task for the Maintainence Prediction agent
maintenance_prediction_task = Task(
    description=(
        "Analyze the vehicle’s usage patterns, odometer readings, oil levels, and historical trends to predict upcoming maintenance needs. "
        "Estimate when the next oil change, tire rotation, or other maintenance is due."
    ),
    expected_output=(
        "Maintenance forecast including estimated due dates for key services like oil changes or tire care."
    ),
    agent=maintenance_predictor
)


In [46]:
# # ### Crew Creation and Execution Cell
# # This cell creates a Crew with all defined agents and tasks, executes the crew,
# # and then displays the final recommendation in a nicely formatted Markdown cell.
def respond_to_vehicle_query1(vehicle_id, user_question, df):
    """Process a vehicle query using CrewAI workflow with fixed tool validation"""
    # Input validation
    if df is None or df.empty:
        return "❌ Please upload vehicle data first."

    if vehicle_id not in df['vehicle_id'].unique():
        return f"❌ Vehicle ID '{vehicle_id}' not found in data."

    try:
        # Create a single tool that handles the vehicle query
        # Important: Use proper Langchain tool format
        from langchain.tools import BaseTool
        from typing import Any, Optional

        class VehicleDataTool(BaseTool):
            name: str = "vehicle_data_tool"  # Add type annotation
            description: str = "Get data about the specified vehicle"  # Add type annotation
            return_direct: bool = False  # Add type annotation

            def _run(self, query: str) -> str:
                vehicle_row = df[df['vehicle_id'] == vehicle_id]
                return vehicle_row.to_string(index=False)

            async def _arun(self, query: str) -> str:
                raise NotImplementedError("This tool does not support async")

        # Create proper tool instances
        vehicle_tool = VehicleDataTool()

        # Create the agents with properly defined tools
        vehicle_specialist = Agent(
            role=vehicle_health_analyst.role,
            goal=vehicle_health_analyst.goal,
            backstory=f"{vehicle_health_analyst.backstory} Currently analyzing vehicle {vehicle_id} to answer: {user_question}",
            llm=llm,
            verbose=True
        )

        alert_specialist = Agent(
            role=alert_generator.role,
            goal=alert_generator.goal,
            backstory=f"{alert_generator.backstory} Currently analyzing vehicle {vehicle_id} to answer: {user_question}",
            llm=llm,
            verbose=True
        )

        fuel_specialist = Agent(
            role=fuel_efficiency_advisor.role,
            goal=fuel_efficiency_advisor.goal,
            backstory=f"{fuel_efficiency_advisor.backstory} Currently analyzing vehicle {vehicle_id} to answer: {user_question}",
            llm=llm,
            verbose=True
        )

        maintenance_specialist = Agent(
            role=maintenance_predictor.role,
            goal=maintenance_predictor.goal,
            backstory=f"{maintenance_predictor.backstory} Currently analyzing vehicle {vehicle_id} to answer: {user_question}",
            llm=llm,
            verbose=True
        )

        # Create tasks with context added to the descriptions
        task1 = Task(
            description=f"Analyze vehicle {vehicle_id} data to answer: '{user_question}'. Examine engine temperature, oil level, tire pressure, speed, and engine load.",
            expected_output="A detailed diagnostic summary answering the user's question.",
            agent=vehicle_specialist
        )

        task2 = Task(
            description=f"For vehicle {vehicle_id}, identify any urgent issues or warnings related to the question: '{user_question}'.",
            expected_output="A list of alerts with severity levels relevant to the user's question.",
            agent=alert_specialist
        )

        task3 = Task(
            description=f"Evaluate the fuel efficiency metrics for vehicle {vehicle_id} in relation to: '{user_question}'.",
            expected_output="Fuel efficiency insights and recommendations relevant to the question.",
            agent=fuel_specialist
        )

        task4 = Task(
            description=f"Predict maintenance needs for vehicle {vehicle_id} based on the data and question: '{user_question}'.",
            expected_output="Maintenance timeline and recommendations addressing the user's question.",
            agent=maintenance_specialist
        )

        # Create the crew with all agents and tasks
        crew = Crew(
            agents=[
                vehicle_specialist,
                alert_specialist,
                fuel_specialist,
                maintenance_specialist
            ],
            tasks=[task1, task2, task3, task4],
            verbose=False,
            process=Process.sequential
        )

        # Run the crew and return results
        result = crew.kickoff()
        return result.raw

    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        return f"❌ Error: {str(e)}\n\nDetails: {error_details}"

In [47]:
import gradio as gr
import pandas as pd
import geocoder
import requests
import asyncio
import matplotlib.pyplot as plt
import seaborn as sns

# --------- Data Functions ---------
global_df = pd.DataFrame()

def load_data_from_file(file):
    try:
        df = pd.read_csv(file.name)
        return "✅ File uploaded successfully.", df
    except Exception as e:
        return f"❌ Error: {e}", None

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")  # Make sure you set this securely

def get_today_weather_advice():
    # 1. Get current location
    g = geocoder.ip("me")
    if not g.ok:
        return "❌ Failed to detect location."

    latitude, longitude = g.latlng
    city, country = g.city, g.country

    # 2. Get today's weather from Open-Meteo
    weather_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": "temperature_2m_max,temperature_2m_min,precipitation_sum",
        "timezone": "auto"
    }

    response = requests.get(weather_url, params=params)
    if response.status_code != 200:
        return "❌ Weather data fetch failed."

    data = response.json()
    today_index = 0
    date = data["daily"]["time"][today_index]
    max_temp = data["daily"]["temperature_2m_max"][today_index]
    min_temp = data["daily"]["temperature_2m_min"][today_index]
    precipitation = data["daily"]["precipitation_sum"][today_index]

    # 3. Format prompt for Gemini
    weather_summary = (
        f"Today's weather for a vehicle in {city}, {country} on {date}:\n"
        f"- Max Temp: {max_temp}°C\n"
        f"- Min Temp: {min_temp}°C\n"
        f"- Precipitation: {precipitation}mm\n\n"
        "As an AI vehicle assistant, You are an expert vehicle assistant. Give concise, smart driving and maintenance advice "
        "for a driver based on this forecast. This is not an EV Vehicle! Format the responses well."
    )

    # 4. Send to Gemini API
    gemini_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={GEMINI_API_KEY}"
    headers = {"Content-Type": "application/json"}
    payload = {
        "contents": [
            {
                "parts": [{"text": weather_summary}]
            }
        ]
    }

    gemini_response = requests.post(gemini_url, headers=headers, json=payload)

    if gemini_response.status_code != 200:
        return f"❌ Gemini API Error: {gemini_response.text}"

    advice = gemini_response.json()["candidates"][0]["content"]["parts"][0]["text"]

    # 5. Return final result
    return f"📍 {city}, {country} — {date}\n🌡 Max: {max_temp}°C | Min: {min_temp}°C\n☔ Precip: {precipitation}mm\n\n🤖 Friendly Advice:\n{advice}"

# --------- Dashboard Summary ---------
def auto_refresh_dashboard(vehicle_id, df):
    if df is None or df.empty:
        return "❌ Please upload data first.", "", "", "", "", "", ""

    if vehicle_id not in df["vehicle_id"].unique():
        return f"❌ Vehicle ID '{vehicle_id}' not found.", "", "", "", "", "", ""

    filtered = df[df["vehicle_id"] == vehicle_id].copy()
    filtered["tracking_timestamp"] = pd.to_datetime(filtered["tracking_timestamp"], dayfirst=True, errors="coerce")
    filtered = filtered.dropna(subset=["tracking_timestamp"])

    for col in [
        "engine_load_percent", "avg_speed_kmph", "temp_engine_C", "fuel_level_percent",
        "tire_pressure_FL", "tire_pressure_FR", "tire_pressure_RL", "tire_pressure_RR"
    ]:
        filtered[col] = pd.to_numeric(filtered[col], errors="coerce")

    filtered = filtered.dropna()
    latest = filtered.iloc[-1]

    temp = f"{latest.get('temp_engine_C', 'N/A')} °C"
    fuel = f"{latest.get('fuel_level_percent', 'N/A')} %"
    tire_fl = f"{latest.get('tire_pressure_FL', 'N/A')} psi"
    tire_fr = f"{latest.get('tire_pressure_FR', 'N/A')} psi"
    tire_rl = f"{latest.get('tire_pressure_RL', 'N/A')} psi"
    tire_rr = f"{latest.get('tire_pressure_RR', 'N/A')} psi"

    return f"✅ Live data for {vehicle_id}", temp, fuel, tire_fl, tire_fr, tire_rl, tire_rr

# --------- Chart 1: Tyre Pressure ---------
def plot_avg_tire_pressure(df):
    try:
        tires = ["tire_pressure_FL", "tire_pressure_FR", "tire_pressure_RL", "tire_pressure_RR"]
        df[tires] = df[tires].apply(pd.to_numeric, errors="coerce")
        avg_pressures = df[tires].mean()

        plt.figure(figsize=(6, 4))
        avg_pressures.plot(kind="bar", color="skyblue")
        plt.title("Average Tyre Pressure")
        plt.ylabel("PSI")
        plt.xticks(rotation=0)
        plt.tight_layout()

        path = "avg_tire_pressure_bar.png"
        plt.savefig(path, bbox_inches="tight")
        plt.close()
        return gr.update(value=path)
    except Exception as e:
        return gr.update(value=None, label=f"❌ Bar Chart Error: {e}")

# --------- Chart 2: Engine Load vs Speed Line ---------
def plot_engine_vs_speed(df):
    try:
        df["tracking_timestamp"] = pd.to_datetime(df["tracking_timestamp"], dayfirst=True, errors="coerce")
        df["engine_load_percent"] = pd.to_numeric(df["engine_load_percent"], errors="coerce")
        df["avg_speed_kmph"] = pd.to_numeric(df["avg_speed_kmph"], errors="coerce")

        df = df.dropna(subset=["tracking_timestamp", "engine_load_percent", "avg_speed_kmph"])
        df = df.sort_values("tracking_timestamp")

        plt.figure(figsize=(10, 5))
        plt.plot(df["tracking_timestamp"], df["engine_load_percent"], label="Engine Load (%)", linewidth=2)
        plt.plot(df["tracking_timestamp"], df["avg_speed_kmph"], label="Average Speed (km/h)", linewidth=2)
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.title("Engine Load vs. Average Speed Over Time")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        path = "engine_vs_speed_linechart.png"
        plt.savefig(path, bbox_inches="tight")
        plt.close()
        return gr.update(value=path)
    except Exception as e:
        return gr.update(value=None, label=f"❌ Line Chart Error: {e}")

# --------- Chart 3: Vehicle Health Overview (Area Chart) ---------
def plot_vehicle_health_area(df):
    try:
        df["tracking_timestamp"] = pd.to_datetime(df["tracking_timestamp"], dayfirst=True, errors="coerce")
        for col in ["temp_engine_C", "fuel_level_percent", "engine_load_percent"]:
            df[col] = pd.to_numeric(df[col], errors="coerce")

        df = df.dropna(subset=["tracking_timestamp", "temp_engine_C", "fuel_level_percent", "engine_load_percent"])
        df = df.set_index("tracking_timestamp").sort_index()

        plt.figure(figsize=(10, 5))
        plt.stackplot(
            df.index,
            df["temp_engine_C"],
            df["fuel_level_percent"],
            df["engine_load_percent"],
            labels=["Engine Temp (°C)", "Fuel Level (%)", "Engine Load (%)"],
            alpha=0.85
        )
        plt.legend(loc="upper right")
        plt.title("Vehicle Health Overview Over Time")
        plt.ylabel("Sensor Value")
        plt.xlabel("Date")
        plt.tight_layout()

        path = "vehicle_health_area_chart.png"
        plt.savefig(path, bbox_inches="tight")
        plt.close()
        return gr.update(value=path)
    except Exception as e:
        return gr.update(value=None, label=f"❌ Area Chart Error: {e}")
# --------- Gradio Interface ---------
with gr.Blocks() as demo:
    gr.Markdown("## 🚗 VoltAI: Smart Vehicle Assistant")

    df_state = gr.State()

    with gr.Tab("🔧 Diagnostics & Upload"):
        with gr.Row():
            file_input = gr.File(label="Upload vehicle_data.csv")
            upload_output = gr.Textbox(label="Upload Status")
        file_input.change(fn=load_data_from_file, inputs=file_input, outputs=[upload_output, df_state])

        with gr.Row():
            vehicle_id_input = gr.Textbox(label="Enter Vehicle ID")
            diagnose_btn = gr.Button("Run Diagnostic")
            diagnosis_output = gr.Textbox(label="Diagnosis Summary", lines=6)

        diagnose_btn.click(fn=generate_vehicle_summary, inputs=[vehicle_id_input, df_state], outputs=diagnosis_output)

    with gr.Tab("❓ Ask About Data"):
      with gr.Row():
        vehicle_id_input = gr.Textbox(label="Vehicle ID (e.g., VH200001)")
        question_input = gr.Textbox(label="Ask a question about recent data")
        ask_btn = gr.Button("Ask")
        answer_output = gr.Textbox(label="AI Answer", lines=8)
      # This is the critical part - make sure the inputs and outputs match your function
      ask_btn.click(
        fn=respond_to_vehicle_query1,
        inputs=[vehicle_id_input, question_input, df_state],  # Make sure df_state is passed as the third argument
        outputs=answer_output)

    with gr.Tab("🌦️ Weather-Based Advice"):
        weather_btn = gr.Button("Get Today's Vehicle Weather Advice")
        weather_output = gr.Textbox(label="Weather + Advice", lines=10)
        weather_btn.click(fn=get_today_weather_advice, outputs=weather_output)

    with gr.Tab("📊 Live Vehicle Dashboard"):
        live_id = gr.Textbox(label="Vehicle ID for Live Monitoring")
        refresh_btn = gr.Button("Refresh Now")
        live_status = gr.Textbox(label="Live Status")

        with gr.Row():
            with gr.Column():
                temp = gr.Textbox(label="Engine Temp")
                fuel = gr.Textbox(label="Fuel Level")
            with gr.Column():
                tire_fl = gr.Textbox(label="Tyre Pressure - Front Left")
                tire_fr = gr.Textbox(label="Tyre Pressure - Front Right")
                tire_rl = gr.Textbox(label="Tyre Pressure - Rear Left")
                tire_rr = gr.Textbox(label="Tyre Pressure - Rear Right")

        with gr.Row():
          weather_btn = gr.Button("Get Today's Vehicle Weather Advice")
          weather_output = gr.Textbox(label="Weather + Advice", lines=10)
          weather_btn.click(fn=get_today_weather_advice, outputs=weather_output)

        with gr.Row():
          vehicle_id_input = gr.Textbox(label="Vehicle ID (e.g., VH200001)")
          question_input = gr.Textbox(label="Ask a question about recent data")
          ask_btn = gr.Button("Ask")
          answer_output = gr.Textbox(label="AI Answer", lines=8)
        # This is the critical part - make sure the inputs and outputs match your function
        ask_btn.click(
          fn=respond_to_vehicle_query1,
          inputs=[vehicle_id_input, question_input, df_state],  # Make sure df_state is passed as the third argument
          outputs=answer_output)

        refresh_btn.click(
            fn=auto_refresh_dashboard,
            inputs=[live_id, df_state],
            outputs=[live_status, temp, fuel, tire_fl, tire_fr, tire_rl, tire_rr]
        )

        with gr.Row():
            show_bar = gr.Button("📊 Avg Tyre Pressure")
            show_line = gr.Button("📈 Engine Load vs Speed")
            show_area = gr.Button("🧠 Vehicle Health Overview")

        with gr.Row():
            bar_out = gr.Image(label="Bar Chart", type="filepath")
            line_out = gr.Image(label="Line Chart", type="filepath")
            area_out = gr.Image(label="Health Overview", type="filepath")

        show_bar.click(fn=plot_avg_tire_pressure, inputs=df_state, outputs=bar_out)
        show_line.click(fn=plot_engine_vs_speed, inputs=df_state, outputs=line_out)
        show_area.click(fn=plot_vehicle_health_area, inputs=df_state, outputs=area_out)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9805d71a266a1c861c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
